# 补充添加之前已入库的内容没有embedding

In [ ]:
import os
import requests
import json
import faiss
import numpy as np
import mysql.connector
from tqdm import tqdm

In [ ]:
OPENAI_KEY = os.getenv('OPENAI_KEY', 'sk-V8RTUXs5GF4dQ5o4B0618fA095F9425986732423563115Ea')
OPENAI_BASE_URL = os.getenv('OPENAI_URL', 'https://api.bltcy.ai/v1')
MODEL_NAME = "BAAI/bge-large-en-v1.5" #"gpt-4o-2024-11-20"

url = f"{OPENAI_BASE_URL}/embeddings"

headers = {
   'Authorization': f'Bearer {OPENAI_KEY}',
   'Content-Type': 'application/json'
}

recall_url = "http://152.32.218.226:9999/news_server/api/recallTask"

In [ ]:
conn = mysql.connector.connect(host="152.32.218.226", user="work", password="Sqe3u8N_VP", database="spider_news")
cursor = conn.cursor()
cursor.execute("SELECT title, feature, unique_id FROM news_detail")

for row in tqdm(cursor):
    title, feature, title_id = row
    try:
        tmp_feature = np.frombuffer(feature, dtype=np.float64).astype('float64')
    except:
        tmp_feature = []
    if feature is None:# or len(tmp_feature)!=1024:
        payload = json.dumps({"model": "BAAI/bge-large-en-v1.5", "input": title})
        
        response = requests.request("POST", url, headers=headers, data=payload).json()
        vector = response['data'][0]['embedding']
        ans = np.array(vector).astype('float64').tolist()
        data = {
            "taskname": "genVec",
            "data": [
                {"id": title_id, "result": ans},
            ]
        }
        call_response = requests.post(recall_url, json=data)
        print(call_response.json())

# 测试mysql

In [ ]:
conn = mysql.connector.connect(host="152.32.218.226", user="work", password="Sqe3u8N_VP", database="spider_news")
cursor = conn.cursor()
cursor.execute("SELECT unique_id, feature FROM news_detail")
features = []
for row in cursor:
    try:
        tmp_feature = np.frombuffer(row[1], dtype=np.float64).astype('float64')
        if len(tmp_feature) == 4096:
            features.append(tmp_feature)
            print(f"有效数据行: {row[0]} - {len(tmp_feature)}")
        else:
            print(f"跳过无效数据行: {row[0]} - {len(tmp_feature)}")
    except Exception as e:
        print(f"跳过无效数据行: {row[0]} - {e}")

In [ ]:
len(features)